In [1]:
import os
import pickle
import gc
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

In [2]:
class Config(object):
    embeddingSize = 256
    sequenceLength = 150
    #sequenceLength = 90

config = Config()

In [ ]:
%%time
is_test = True

click_times = pd.read_pickle('./idlist/click_times_list.pkl')
for col in tqdm(['creative_id', 'ad_id', 'advertiser_id', 'product_id', 'industry','product_category']):    
    list_d = pd.read_pickle('./idlist/{}_list.pkl'.format(col))
    We = np.load('./w2v_256_10/{}_embedding_weight.npy'.format(col))
    We = np.vstack([We, np.zeros(config.embeddingSize)])
    list_d = list(list_d)
    for i in range(len(list_d)):
        ret = []
        for j in range(len(list_d[i])):
            ret += [list_d[i][j]] * click_times[i][j]
        list_d[i] = ret      
        
        if len(list_d[i]) > config.sequenceLength:
            list_d[i] = list_d[i][:config.sequenceLength]
        else:
            list_d[i] += [len(We)-1] * (config.sequenceLength - len(list_d[i]))
    list_d = np.array(list_d)
    list_d = list_d.astype(np.int32) #减少内存使用量
    np.save('./inputs_new/{}_inputs_train.npy'.format(col), list_d[:3000000])
    np.save('./inputs_new/{}_inputs_test.npy'.format(col), list_d[3000000:])
    del list_d
    gc.collect()